In [1]:
# Importing useful libraries
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import  scipy.misc
import glob
import os
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding1D, BatchNormalization, Flatten, Conv1D
from keras.layers import AveragePooling1D, MaxPooling1D, Dropout, GlobalMaxPooling1D, GlobalAveragePooling1D 
from keras.models import Model, Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
 
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import keras.backend as K

In [2]:
# Loading the data
train=np.empty(shape=(530,200,250))
y=[]
for filename in glob.iglob('/Users/siddhantbansal/Desktop/Diab_global/*.*'):
    print(filename)
    img=scipy.misc.imread(filename,mode='L')
    img=scipy.misc.imresize(img,(200,250))
    train[counter]=img
    y.append(1)
for filename in glob.iglob('/Users/siddhantbansal/Desktop/Non_Diabetes/*.*'):
    print(filename)
    img=scipy.misc.imread(filename,mode='L')
    img=scipy.misc.imresize(img,(200,250))
    train[counter]=img
    y.append(0)

/Users/siddhantbansal/Desktop/Diab_global/L1'.png
/Users/siddhantbansal/Desktop/Diab_global/L1.png
/Users/siddhantbansal/Desktop/Diab_global/L10'.png
/Users/siddhantbansal/Desktop/Diab_global/L10.png
/Users/siddhantbansal/Desktop/Diab_global/L100'.png
/Users/siddhantbansal/Desktop/Diab_global/L100.png
/Users/siddhantbansal/Desktop/Diab_global/L101'.png
/Users/siddhantbansal/Desktop/Diab_global/L101.png
/Users/siddhantbansal/Desktop/Diab_global/L102'.png
/Users/siddhantbansal/Desktop/Diab_global/L102.png
/Users/siddhantbansal/Desktop/Diab_global/L103'.png
/Users/siddhantbansal/Desktop/Diab_global/L103.png
/Users/siddhantbansal/Desktop/Diab_global/L104'.png
/Users/siddhantbansal/Desktop/Diab_global/L104.png
/Users/siddhantbansal/Desktop/Diab_global/L105'.png
/Users/siddhantbansal/Desktop/Diab_global/L105.png
/Users/siddhantbansal/Desktop/Diab_global/L106'.png
/Users/siddhantbansal/Desktop/Diab_global/L106.png
/Users/siddhantbansal/Desktop/Diab_global/L107'.png
/Users/siddhantbansal/Deskt

/Users/siddhantbansal/Desktop/Diab_global/L47.png
/Users/siddhantbansal/Desktop/Diab_global/L48'.png
/Users/siddhantbansal/Desktop/Diab_global/L48.png
/Users/siddhantbansal/Desktop/Diab_global/L49'.png
/Users/siddhantbansal/Desktop/Diab_global/L49.png
/Users/siddhantbansal/Desktop/Diab_global/L5'.png
/Users/siddhantbansal/Desktop/Diab_global/L5.png
/Users/siddhantbansal/Desktop/Diab_global/L50'.png
/Users/siddhantbansal/Desktop/Diab_global/L50.png
/Users/siddhantbansal/Desktop/Diab_global/L51'.png
/Users/siddhantbansal/Desktop/Diab_global/L51.png
/Users/siddhantbansal/Desktop/Diab_global/L52'.png
/Users/siddhantbansal/Desktop/Diab_global/L52.png
/Users/siddhantbansal/Desktop/Diab_global/L53'.png
/Users/siddhantbansal/Desktop/Diab_global/L53.png
/Users/siddhantbansal/Desktop/Diab_global/L54'.png
/Users/siddhantbansal/Desktop/Diab_global/L54.png
/Users/siddhantbansal/Desktop/Diab_global/L55'.png
/Users/siddhantbansal/Desktop/Diab_global/L55.png
/Users/siddhantbansal/Desktop/Diab_global/L

/Users/siddhantbansal/Desktop/Non_Diabetes/L167.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L168.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L169.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L17.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L170.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L171.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L172.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L173.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L174.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L175.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L176.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L177.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L178.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L179.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L18.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L180.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L181.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L182.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L183.png
/Users/siddhan

/Users/siddhantbansal/Desktop/Non_Diabetes/L93.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L94.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L95.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L96.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L97.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L98.png
/Users/siddhantbansal/Desktop/Non_Diabetes/L99.png


In [3]:
train = train/255.0

In [4]:
y=np.array(y)
y=y.reshape(530,1)

In [5]:
#Perfect, now having train and test we also need to shuffle the data as a good practice for evaluation and fitting 
from random import shuffle
N=train.shape[0]
ind_list = [i for i in range(N)]
shuffle(ind_list)

train_new  = train[ind_list, :,:]
target_new = y[ind_list,]
train=train_new
y=target_new

In [8]:
scores = []
for i in range(50):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=i)

In [11]:
def DocumentModel(input_shape):
    """
    Implementation of the DocumentModel.
    
    Arguments:
    input_shape -- shape of the images of the dataset
    Returns:
    model -- a Model() instance in Keras
    """
    
 
   
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)
    X=Sequential()

   
   
 
    # CONV -> BN -> RELU Block applied to X
    X = Conv1D(32, 7, strides = 1, name = 'conv1')(X_input)
    X = BatchNormalization(axis = 2, name = 'bn1')(X)
    X = Activation('relu')(X)
    X = MaxPooling1D(2, name='max_pool1')(X)
    
 
   
    # CONV -> BN -> RELU Block applied to X
    X = Conv1D(64, 5, strides = 1, name = 'conv2')(X)
    X = BatchNormalization(axis = 2, name = 'bn2')(X)
    X = Activation('relu')(X)
    X = MaxPooling1D(2, name='max_pool2')(X)
    
 
 
    # CONV -> BN -> RELU Block applied to X
    X = Conv1D(128, 3, strides = 1, name = 'conv3')(X)
    X = BatchNormalization(axis = 2, name = 'bn3')(X)
    X = Activation('relu')(X)
    X = MaxPooling1D(2, name='max_pool3')(X)
    
 
    # CONV -> BN -> RELU Block applied to X
    X = Conv1D(64, 1, strides = 1, name = 'conv4')(X)
    X = BatchNormalization(axis = 2, name = 'bn4')(X)
    X = Activation('relu')(X)

    #layer group5 4*4*32
  
 
    X = Conv1D(32, 3, strides = 1, name = 'conv5')(X)
    X = BatchNormalization(axis = 2, name = 'bn5')(X)
    X = Activation('relu')(X)
    X = MaxPooling1D(2, name='max_pool5')(X)
    
    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(128, activation='sigmoid', name='fc1')(X)
    X = Dense(32, activation='sigmoid', name='fc2')(X)
    X = Dense(2, activation='sigmoid', name='fc3')(X)
    
    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='DocumentModel')
    
    ### END CODE HERE ###
    
    return model

happyModel = DocumentModel(((200,250)))
happyModel.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [19]:
scores=[]
 
for i in range(50): 
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=i)
    happyModel = DocumentModel(((200,250)))
    happyModel.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
    from keras.utils.np_utils import to_categorical
    categorical_labels = to_categorical(y_train, num_classes=None)
    %time happyModel.fit(x = X_train, y = categorical_labels, epochs = 20, batch_size = 32)
    pred=happyModel.predict(X_test)
    y_classes = pred.argmax(axis=-1)
    from sklearn.metrics import   accuracy_score
    scores.append(accuracy_score(y_classes,y_test))
    print('accuracy ' , accuracy_score(y_classes,y_test))

Epoch 1/20
424/424 [==============================] - 45s 107ms/step - loss: 0.5971 - acc: 0.7854
Epoch 2/20
424/424 [==============================] - 2s 6ms/step - loss: 0.3230 - acc: 0.9245
Epoch 3/20
424/424 [==============================] - 2s 6ms/step - loss: 0.2279 - acc: 0.9269
Epoch 4/20
424/424 [==============================] - 2s 6ms/step - loss: 0.1724 - acc: 0.9552
Epoch 5/20
424/424 [==============================] - 2s 6ms/step - loss: 0.1297 - acc: 0.9717
Epoch 6/20
424/424 [==============================] - 2s 6ms/step - loss: 0.1136 - acc: 0.9764
Epoch 7/20
424/424 [==============================] - 2s 6ms/step - loss: 0.1062 - acc: 0.9811
Epoch 8/20
424/424 [==============================] - 2s 6ms/step - loss: 0.1264 - acc: 0.9646
Epoch 9/20
424/424 [==============================] - 2s 6ms/step - loss: 0.0975 - acc: 0.9741
Epoch 10/20
424/424 [==============================] - 2s 6ms/step - loss: 0.0890 - acc: 0.9764
Epoch 11/20
424/424 [=========================

In [15]:
print(len(scores))

50


In [16]:
print(np.mean(scores))
# Mean for 150 equal cases was 0.7898245614035088
# Mean for 250+ equal cases was 0.768679245283019 For first run
# Mean for 250+ equal cases was 0.7767924528301886 For second run

0.7767924528301886


In [17]:
print(np.std(scores))
# Standard deviation for 150 equal cases was 0.15934963233316635
# Standard deviation for 250+ equal cases was 0.12086139156114381
# Standard deviation for 250+ equal cases was 0.12292579532789435

0.12292579532789435


In [10]:
print(scores)

[0.6792452830188679, 0.8018867924528302, 0.7735849056603774, 0.5849056603773585, 0.8584905660377359, 0.7735849056603774, 0.8113207547169812, 0.6698113207547169, 0.9245283018867925, 0.8962264150943396, 0.8679245283018868, 0.5094339622641509, 0.7547169811320755, 0.7641509433962265, 0.6132075471698113, 0.8490566037735849, 0.8113207547169812, 0.9150943396226415, 0.5377358490566038, 0.9150943396226415, 0.8490566037735849, 0.7452830188679245, 0.9056603773584906, 0.7169811320754716, 0.4716981132075472, 0.8584905660377359, 0.7830188679245284, 0.5283018867924528, 0.7830188679245284, 0.7358490566037735, 0.6981132075471698, 0.9245283018867925, 0.8207547169811321, 0.9150943396226415, 0.8584905660377359, 0.8490566037735849, 0.8490566037735849, 0.8679245283018868, 0.6792452830188679, 0.7452830188679245, 0.8301886792452831, 0.8113207547169812, 0.8018867924528302, 0.6698113207547169, 0.8679245283018868, 0.5660377358490566, 0.8490566037735849, 0.8773584905660378, 0.5283018867924528, 0.7358490566037735]

In [18]:
print(scores)

[0.7735849056603774, 0.7924528301886793, 0.7830188679245284, 0.8584905660377359, 0.8301886792452831, 0.8301886792452831, 0.5660377358490566, 0.5660377358490566, 0.5660377358490566, 0.8962264150943396, 0.7358490566037735, 0.6132075471698113, 0.6132075471698113, 0.9150943396226415, 0.8679245283018868, 0.6792452830188679, 0.7169811320754716, 0.8301886792452831, 0.7358490566037735, 0.8867924528301887, 0.8679245283018868, 0.6886792452830188, 0.5283018867924528, 0.9528301886792453, 0.5188679245283019, 0.9339622641509434, 0.7641509433962265, 0.7452830188679245, 0.8773584905660378, 0.8584905660377359, 0.839622641509434, 0.6037735849056604, 0.8962264150943396, 0.7547169811320755, 0.6037735849056604, 0.8962264150943396, 0.5, 0.8773584905660378, 0.9339622641509434, 0.8773584905660378, 0.9339622641509434, 0.7830188679245284, 0.8113207547169812, 0.839622641509434, 0.8207547169811321, 0.8490566037735849, 0.8584905660377359, 0.8490566037735849, 0.7358490566037735, 0.7830188679245284]
